In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import wrds
import psycopg2 
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats

## wrds

In [ ]:
import wrds
db = wrds.Connection()
data = db.raw_sql('SELECT date,dji FROM djones.djdaily')

In [ ]:
import pandas as pd
df = pd.DataFrame(data)
df.set_index('date', inplace = True)
df.head()

In [ ]:
import matplotlib.pyplot as plt
#df['date'] = pd.to_datetime(df.index.get_level_values(0), format = '%Y%m')
df[['dji']].plot()
#plt.show()

In [ ]:
#db.list_libraries()

In [ ]:
#db.list_tables(library="crsp")

In [ ]:
#db.list_tables(library="comp")

In [ ]:
db.describe_table(library="crsp", table="crsp_daily_data")

In [ ]:
data = db.get_table(library='djones', table='djdaily', columns=['date', 'dji'], obs=10)
data

In [ ]:
import wrds
import pandas as pd
db = wrds.Connection()

In [ ]:
comp_name = db.get_table(library = 'comp', table = 'names')

In [ ]:
comp_name.to_pickle('comp_name.pkl')

In [ ]:
crsp_name = db.get_table(library = 'crsp', table = 'msenames')

In [ ]:
db.describe_table('crsp', 'msf')

In [ ]:
db.describe_table('crsp', 'msenames')

In [ ]:
db.describe_table('comp', 'security')

In [ ]:
crsp_msf = db.get_table(library = 'crsp', table = 'msf', columns=['permno','cusip', 'vol', 'prc','ret'])

In [ ]:
crsp_msf.to_pickle('crsp_msf.pkl')

# merge comp and crsp

In [11]:
crsp_msf = pd.read_pickle('crsp_msf.pkl')

In [2]:
comp_name = pd.read_pickle('comp_name.pkl')

In [12]:
crsp_msf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4413466 entries, 0 to 4413465
Data columns (total 5 columns):
permno    float64
cusip     object
vol       float64
prc       float64
ret       float64
dtypes: float64(4), object(1)
memory usage: 168.4+ MB


In [13]:
crsp_msf.head()

,permno,cusip,vol,prc,ret
0,10000.0,68391610,NaN,NaN,NaN
1,10000.0,68391610,1771.0,-4.3750,NaN
2,10000.0,68391610,828.0,-3.2500,-0.257143
3,10000.0,68391610,1078.0,-4.4375,0.365385
4,10000.0,68391610,957.0,-4.0000,-0.098592


In [3]:
comp_name.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38728 entries, 0 to 38727
Data columns (total 9 columns):
gvkey    38728 non-null object
conm     38728 non-null object
tic      38704 non-null object
cusip    38710 non-null object
cik      28506 non-null object
sic      38728 non-null object
naics    34199 non-null object
year1    38728 non-null float64
year2    38728 non-null float64
dtypes: float64(2), object(7)
memory usage: 2.7+ MB


In [4]:
comp_name.head()

,gvkey,conm,tic,cusip,cik,sic,naics,year1,year2
0,001000,A & E PLASTIK PAK INC,AE.2,000032102,None,3089,None,1961.0,1977.0
1,001001,A & M FOOD SERVICES INC,AMFD.,000165100,0000723576,5812,722,1978.0,1985.0
2,001002,AAI CORP,AAIC,000352104,0000001745,3825,None,1960.0,1972.0
3,001003,A.A. IMPORTING CO INC,ANTQ,000354100,0000730052,5712,442110,1980.0,1989.0
4,001004,AAR CORP,AIR,000361105,0000001750,5080,423860,1965.0,2016.0


In [8]:
type(comp_name['cusip'])

pandas.core.series.Series

In [9]:
comp_name['cusip_8'] = comp_name['cusip'].str[:8]

In [10]:
comp_name.head()

,gvkey,conm,tic,cusip,cik,sic,naics,year1,year2,cusip_8
0,001000,A & E PLASTIK PAK INC,AE.2,000032102,None,3089,None,1961.0,1977.0,00003210
1,001001,A & M FOOD SERVICES INC,AMFD.,000165100,0000723576,5812,722,1978.0,1985.0,00016510
2,001002,AAI CORP,AAIC,000352104,0000001745,3825,None,1960.0,1972.0,00035210
3,001003,A.A. IMPORTING CO INC,ANTQ,000354100,0000730052,5712,442110,1980.0,1989.0,00035410
4,001004,AAR CORP,AIR,000361105,0000001750,5080,423860,1965.0,2016.0,00036110


In [23]:
comp_name = comp_name.set_index('cusip_8')

In [24]:
crsp_msf = crsp_msf.set_index('cusip')

In [26]:
ccm = pd.merge(comp_name, crsp_msf, left_index = True, right_index= True)

In [27]:
#ccm1 = pd.concat([comp_name, crsp_msf], axis =1 )

In [28]:
ccm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3588403 entries, 00002130 to Y9530810
Data columns (total 13 columns):
gvkey     object
conm      object
tic       object
cusip     object
cik       object
sic       object
naics     object
year1     float64
year2     float64
permno    float64
vol       float64
prc       float64
ret       float64
dtypes: float64(6), object(7)
memory usage: 383.3+ MB


In [30]:
ccm

,gvkey,conm,tic,cusip,cik,sic,naics,year1,year2,permno,vol,prc,ret
00002130,023052,A & A INTL INDS INC,3ANAFE,000021303,0000855569,2020,311513,1984.0,1999.0,78736.0,NaN,NaN,NaN
00002130,023052,A & A INTL INDS INC,3ANAFE,000021303,0000855569,2020,311513,1984.0,1999.0,78736.0,78.0,-3.125000,NaN
00002130,023052,A & A INTL INDS INC,3ANAFE,000021303,0000855569,2020,311513,1984.0,1999.0,78736.0,334.0,-2.875000,-0.080000
00002130,023052,A & A INTL INDS INC,3ANAFE,000021303,0000855569,2020,311513,1984.0,1999.0,78736.0,624.0,-2.843750,-0.010870
00002130,023052,A & A INTL INDS INC,3ANAFE,000021303,0000855569,2020,311513,1984.0,1999.0,78736.0,465.0,-2.625000,-0.076923
00002130,023052,A & A INTL INDS INC,3ANAFE,000021303,0000855569,2020,311513,1984.0,1999.0,78736.0,622.0,-2.187500,-0.166667
00002130,023052,A & A INTL INDS INC,3ANAFE,000021303,0000855569,2020,311513,1984.0,1999.0,78736.0,660.0,-2.187500,0.000000
00002130,023052,A & A INTL INDS INC,3ANAFE,000021303,0000855569,2020,311513,1984.0,1999.0,78736.0,659.0,-3.000000,0.371429
00002130,023052,A & A INTL INDS INC,3ANAFE,000021303,0000855569,2020,311513,1984.0,1999.0,78736.0,2118.0,-3.062500,0.020833
00002130,023052,A & A INTL INDS INC,3ANAFE,000021303,0000855569,2020,311513,1984.0,1999.0,78736.0,3497.0,-4.875000,0.591837
